# National Bridge Inventory - Prep for "Migration" to SQL Database
### Dependencies

In [1]:
# Initial imports
import pandas as pd
from pathlib import Path

#regex library has functionality beyond the built-in Python re module
import regex as re

### Loading and Preprocessing Bridge Encoded Data



In [2]:
# Loading data
bridge_data = pd.read_csv("Resources/NTAD_National_Bridge_Inventory.csv", low_memory=False)
bridge_data.head()

,OBJECTID,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,...,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA,STATUS,DATE,LATDD,LONGDD,x,y
0,1,1,00000000000S702,1,6,1,04007,0,09,53,...,62,F,5,145.50,ST,723,31.103039,-87.570411,-87.570411,31.103039
1,2,1,00000000000S703,1,6,1,04003,0,09,53,...,62,F,6,133.28,ST,723,31.105611,-87.569139,-87.569139,31.105611
2,3,1,0000000000M0022,1,8,1,00000,0,00,113,...,74,F,5,387.00,ST,723,32.295361,-84.977219,-84.977219,32.295361
3,4,1,000000883039900,1,4,1,00088,0,02,59,...,67,G,7,687.75,AM,1202,34.454228,-87.975517,-87.975517,34.454228
4,5,1,000001014002450,1,3,1,00101,0,02,79,...,67,F,5,15453.36,MM,403,34.813317,-87.382372,-87.382372,34.813317


In [3]:
# Create copy to clean file for SQL import
bridge_data_clean = bridge_data.copy()

# Check field values for one column before transformation
bridge_data_clean['TRANSITIONS_036B'].unique()

array(['1', '0', 'N', nan, 'n'], dtype=object)

In [4]:
# Define a function to replace non-values like 'N' or 'n' with an empty string using regex.fullmatch
def replace_with_null(value):
    if re.fullmatch(r'[/*]', str(value)):
        return ''
    elif re.fullmatch(r'[Nn]',str(value)):
        return ''
    else:
        return value

# List the dataset columns that have encoded non-values    
series_list = ['RAILINGS_036A','TRANSITIONS_036B','APPR_RAIL_036C','APPR_RAIL_END_036D',
               'NAVIGATION_038','DECK_COND_058','SUPERSTRUCTURE_COND_059',
               'SUBSTRUCTURE_COND_060','CHANNEL_COND_061','CULVERT_COND_062',
               'STRUCTURAL_EVAL_067','DECK_GEOMETRY_EVAL_068','UNDCLRENCE_EVAL_069',
               'WATERWAY_EVAL_071','DECK_STRUCTURE_TYPE_107','SURFACE_TYPE_108A',
               'MEMBRANE_TYPE_108B', 'DECK_PROTECTION_108C'
              ]

# remove non-values from the dataset
bridge_data_clean[series_list] = bridge_data_clean[series_list].map(replace_with_null)

##### Note: NaN values may display in Jupyter notebook as string 'nan', but still export as blank.  
##### Make sure to examine the .csv export file to check that the final export displays correctly in Excel/Notepad

In [5]:
bridge_data_clean['TRANSITIONS_036B'].unique()

array(['1', '0', '', nan], dtype=object)

In [6]:
# export clean df to review
bridge_data_clean.to_csv("output_data/bridge_data_clean.csv", index=False, lineterminator=None,sep=',')

In [7]:
# Create version of df with columns in lowercase for SQL import
bridge_data_clean.columns = [x.lower() for x in bridge_data_clean.columns]

# Preview columns
print(bridge_data_clean.columns)

Index(['objectid', 'state_code_001', 'structure_number_008',
       'record_type_005a', 'route_prefix_005b', 'service_level_005c',
       'route_number_005d', 'direction_005e', 'highway_district_002',
       'county_code_003',
       ...
       'submitted_by', 'bridge_condition', 'lowest_rating', 'deck_area',
       'status', 'date', 'latdd', 'longdd', 'x', 'y'],
      dtype='object', length=130)


### Creating Load Sheets for Import to SQL

In [8]:
# export clean df for SQL import
bridge_data_clean.to_csv("output_data/bridge_data_clean.csv", index=False, lineterminator=None,sep=',')

##### Note:  Repeat the cleaning and load steps for each table to create in SQL

### Checking Original Dataset vs. Clean Dataset (Basic Profile and Counts)

In [9]:
# Review original dataframe dimensions
bridge_data.info()
bridge_data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621581 entries, 0 to 621580
Columns: 130 entries, OBJECTID to y
dtypes: float64(46), int64(41), object(43)
memory usage: 616.5+ MB


(621581, 130)

In [10]:
# Review original dataframe stats
bridge_data.describe()

,OBJECTID,STATE_CODE_001,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,DIRECTION_005E,COUNTY_CODE_003,PLACE_CODE_004,CRITICAL_FACILITY_006B,MIN_VERT_CLR_010,...,YEAR_OF_FUTURE_ADT_115,MIN_NAV_CLR_MT_116,SUBMITTED_BY,LOWEST_RATING,DECK_AREA,DATE,LATDD,LONGDD,x,y
count,621581.000000,621581.000000,621581.0,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000,0.0,621581.000000,...,621581.000000,269718.000000,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000
mean,310791.000000,30.017867,1.0,3.564269,1.160912,0.221323,94.867071,21075.474754,NaN,93.612089,...,2018.016585,0.040920,30.690039,6.208558,649.132506,777.806148,37.902957,-91.752495,-91.752495,37.902957
std,179435.123173,15.374181,0.0,1.375336,1.256330,0.675185,90.471975,28851.633603,NaN,21.185916,...,180.484180,3.973447,15.930257,1.161318,2559.737351,294.600972,5.048528,13.059281,13.059281,5.048528
min,1.000000,1.000000,1.0,1.000000,0.000000,0.000000,1.000000,0.000000,NaN,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.920000,203.000000,0.000000,-174.182281,-174.182281,0.000000
25%,155396.000000,18.000000,1.0,3.000000,1.000000,0.000000,33.000000,0.000000,NaN,99.990000,...,2034.000000,0.000000,18.000000,5.000000,94.720000,603.000000,34.409225,-97.150950,-97.150950,34.409225
50%,310791.000000,29.000000,1.0,4.000000,1.000000,0.000000,77.000000,0.000000,NaN,99.990000,...,2040.000000,0.000000,30.000000,6.000000,212.480000,723.000000,38.487453,-90.234969,-90.234969,38.487453
75%,466186.000000,42.000000,1.0,4.000000,1.000000,0.000000,127.000000,41000.000000,NaN,99.990000,...,2041.000000,0.000000,45.000000,7.000000,589.260000,1202.000000,41.316875,-83.006944,-83.006944,41.316875
max,621581.000000,78.000000,1.0,8.000000,8.000000,4.000000,840.000000,99999.000000,NaN,99.990000,...,5038.000000,999.900000,79.000000,9.000000,372399.360000,1202.000000,70.216450,144.820158,144.820158,70.216450


In [11]:
# Review current dataframe dimensions
bridge_data_clean.info()
bridge_data_clean.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621581 entries, 0 to 621580
Columns: 130 entries, objectid to y
dtypes: float64(46), int64(41), object(43)
memory usage: 616.5+ MB


(621581, 130)

In [12]:
# Review current dataframe stats
bridge_data_clean.describe()

,objectid,state_code_001,record_type_005a,route_prefix_005b,service_level_005c,direction_005e,county_code_003,place_code_004,critical_facility_006b,min_vert_clr_010,...,year_of_future_adt_115,min_nav_clr_mt_116,submitted_by,lowest_rating,deck_area,date,latdd,longdd,x,y
count,621581.000000,621581.000000,621581.0,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000,0.0,621581.000000,...,621581.000000,269718.000000,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000,621581.000000
mean,310791.000000,30.017867,1.0,3.564269,1.160912,0.221323,94.867071,21075.474754,NaN,93.612089,...,2018.016585,0.040920,30.690039,6.208558,649.132506,777.806148,37.902957,-91.752495,-91.752495,37.902957
std,179435.123173,15.374181,0.0,1.375336,1.256330,0.675185,90.471975,28851.633603,NaN,21.185916,...,180.484180,3.973447,15.930257,1.161318,2559.737351,294.600972,5.048528,13.059281,13.059281,5.048528
min,1.000000,1.000000,1.0,1.000000,0.000000,0.000000,1.000000,0.000000,NaN,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.920000,203.000000,0.000000,-174.182281,-174.182281,0.000000
25%,155396.000000,18.000000,1.0,3.000000,1.000000,0.000000,33.000000,0.000000,NaN,99.990000,...,2034.000000,0.000000,18.000000,5.000000,94.720000,603.000000,34.409225,-97.150950,-97.150950,34.409225
50%,310791.000000,29.000000,1.0,4.000000,1.000000,0.000000,77.000000,0.000000,NaN,99.990000,...,2040.000000,0.000000,30.000000,6.000000,212.480000,723.000000,38.487453,-90.234969,-90.234969,38.487453
75%,466186.000000,42.000000,1.0,4.000000,1.000000,0.000000,127.000000,41000.000000,NaN,99.990000,...,2041.000000,0.000000,45.000000,7.000000,589.260000,1202.000000,41.316875,-83.006944,-83.006944,41.316875
max,621581.000000,78.000000,1.0,8.000000,8.000000,4.000000,840.000000,99999.000000,NaN,99.990000,...,5038.000000,999.900000,79.000000,9.000000,372399.360000,1202.000000,70.216450,144.820158,144.820158,70.216450


### Analyis to Prepare the Database Schema

In [13]:
# Get column list
print(bridge_data_clean.columns.tolist())

['objectid', 'state_code_001', 'structure_number_008', 'record_type_005a', 'route_prefix_005b', 'service_level_005c', 'route_number_005d', 'direction_005e', 'highway_district_002', 'county_code_003', 'place_code_004', 'features_desc_006a', 'critical_facility_006b', 'facility_carried_007', 'location_009', 'min_vert_clr_010', 'kilopoint_011', 'base_hwy_network_012', 'lrs_inv_route_013a', 'subroute_no_013b', 'lat_016', 'long_017', 'detour_kilos_019', 'toll_020', 'maintenance_021', 'owner_022', 'functional_class_026', 'year_built_027', 'traffic_lanes_on_028a', 'traffic_lanes_und_028b', 'adt_029', 'year_adt_030', 'design_load_031', 'appr_width_mt_032', 'median_code_033', 'degrees_skew_034', 'structure_flared_035', 'railings_036a', 'transitions_036b', 'appr_rail_036c', 'appr_rail_end_036d', 'history_037', 'navigation_038', 'nav_vert_clr_mt_039', 'nav_horr_clr_mt_040', 'open_closed_posted_041', 'service_on_042a', 'service_und_042b', 'structure_kind_043a', 'structure_type_043b', 'appr_kind_044

In [25]:
# Get column datatypes
dict(bridge_data_clean.dtypes)

{'objectid': dtype('int64'),
 'state_code_001': dtype('int64'),
 'structure_number_008': dtype('O'),
 'record_type_005a': dtype('int64'),
 'route_prefix_005b': dtype('int64'),
 'service_level_005c': dtype('int64'),
 'route_number_005d': dtype('O'),
 'direction_005e': dtype('int64'),
 'highway_district_002': dtype('O'),
 'county_code_003': dtype('int64'),
 'place_code_004': dtype('int64'),
 'features_desc_006a': dtype('O'),
 'critical_facility_006b': dtype('float64'),
 'facility_carried_007': dtype('O'),
 'location_009': dtype('O'),
 'min_vert_clr_010': dtype('float64'),
 'kilopoint_011': dtype('float64'),
 'base_hwy_network_012': dtype('float64'),
 'lrs_inv_route_013a': dtype('O'),
 'subroute_no_013b': dtype('float64'),
 'lat_016': dtype('float64'),
 'long_017': dtype('float64'),
 'detour_kilos_019': dtype('int64'),
 'toll_020': dtype('int64'),
 'maintenance_021': dtype('int64'),
 'owner_022': dtype('int64'),
 'functional_class_026': dtype('int64'),
 'year_built_027': dtype('int64'),
 

In [29]:
# Get column unique values
unique_values = {}
for column in bridge_data_clean.columns:
    unique_values[column] = bridge_data_clean[column].unique()
    print(column)
    print(unique_values[column])
    print()



objectid
[     1      2      3 ... 621579 621580 621581]

state_code_001
[ 1  2  4  5  6  8  9 10 11 12 13 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 44 45 46 47 48 49 50 51 53
 54 55 56 72 66 78]

structure_number_008
['00000000000S702' '00000000000S703' '0000000000M0022' ... 'STX7520'
 'STX8001' 'STX8205']

record_type_005a
[1]

route_prefix_005b
[6 8 4 3 5 2 7 1]

service_level_005c
[1 2 8 0 4 3 7 6]

route_number_005d
['04007' '04003' '00000' ... '05553' '04480' '04494']

direction_005e
[0 1 4 3 2]

highway_district_002
['09' '00' '02' '10' '01' '05' '03' '04' '08' '07' '06' '2' '7' '9' '5'
 '4' '6' 'CN' 'N' 'SE' '3' '20' 'GL' 'PH' 'TU' 'SA' 'HO' 'KI' 'PR' '85'
 '87' '90' '60' '78' '84' '81' '82' '88' '8' '11' '12' '1' '54' '69' '40'
 '68' '57' '32' '46' '41' '36' '15' '53' '24' '14' '31' '19' '45' '44'
 '52' '37' '27' '39' '62' '63' '26' '25' '34' '30' '21' nan 'KY' 'IL' 'PV'
 'OH' '70' '51' '23' '13' '42' '22' '55' '33' '61' '43' '58' '16